In [1]:
%cd ..

/home/sheik_dawood/Workspace/compoundGAN


In [2]:
import torch
from torch.utils.data import DataLoader

from data.zinc import ZINC250K, ZINC_collate
from models.gcn import GCN

In [3]:
model = GCN(58, 11, 256, [64, 128, 128])

In [7]:
dataset = ZINC250K('/home/sheik_dawood/store/Datasets/ZINC250K/data.csv')

In [8]:
dataloader = DataLoader(dataset, 8, shuffle=True, collate_fn=ZINC_collate)

In [9]:
for i, (g, atom_y, bond_y) in enumerate(dataloader):
    break
atom_y.shape

torch.Size([62, 8, 5])

In [16]:
bond_y[2].shape

torch.Size([8, 2, 4])

In [14]:
g.ndata['feats'].shape

torch.Size([346, 59])

In [9]:
model(g).shape

RuntimeError: size mismatch, m1: [367 x 72], m2: [69 x 64] at /Users/distiller/project/conda/conda-bld/pytorch_1565272526878/work/aten/src/TH/generic/THTensorMath.cpp:752

In [ ]:
torch.Tensor(atom_y[0]).shape

In [ ]:
mask.shape

In [ ]:
len(atom_y)